In [28]:
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from tqdm import tqdm

import multiprocessing
import os
import pydicom

import json

## Convert DCM to PNG

In [3]:
with open("../SETTINGS.json") as f:
    SETTINGS = json.load(f)
    
SETTINGS

{'TRAIN_RAW_DATA_DIR': 'data/stage_2_train_images',
 'TEST_RAW_DATA_DIR': 'data/stage_2_test_images',
 'TRAIN_CLEAN_DATA_DIR': 'data/stage_2_train_images_png',
 'TEST_CLEAN_DATA_DIR': 'data/stage_2_test_images_png',
 'TRAINING_SPLIT_DIR': 'training_splits',
 'DETAILED_CLASS_INFO': 'data/stage_2_detailed_class_info.csv',
 'TRAIN_LABELS': 'data/stage_2_train_labels.csv',
 'CLASSIFICATION_WEIGHTS_DIR': 'classification/weights',
 'CLASSIFICATION_RESULTS_DIR': 'classification/results',
 'DETECTION_WEIGHTS_DIR': 'detection/weights',
 'DETECTION_RESULTS_DIR': 'detection/results'}

In [4]:
# os.chdir(os.path.dirname(__file__))
print(os.getcwd())

/home/arif/Arif/Private/kaggle/pneumonia/rcnn/preprocessing


In [30]:
SRC_PATH = os.path.join("../", SETTINGS["TRAIN_RAW_DATA_DIR"])
DST_PATH  = os.path.join("../", SETTINGS["TRAIN_CLEAN_DATA_DIR"])
print(SRC_PATH)
print(DST_PATH)


../data/stage_2_train_images
../data/stage_2_train_images_png


In [20]:
def convert(patientId, src_path=SRC_PATH, dst_path=DST_PATH):
    '''
    Convert dcm to png
    '''
    Image.fromarray(
        pydicom.dcmread(
            os.path.join(SRC_PATH, patientId + '.dcm')
        ).pixel_array
    ).save(os.path.join(DST_PATH, patientId + '.png'))

In [29]:
def main():
    '''
    Main Method
    '''
    assert os.path.lexists(SRC_PATH)
    assert os.path.lexists(DST_PATH)

    patient_ids = [os.path.splitext(f)[0] for f in os.listdir(SRC_PATH)]

    with multiprocessing.Pool(multiprocessing.cpu_count()) as p:
        pbar = tqdm(
              total=len(patient_ids)
            , leave=False
            , desc="Converting"
        )
        for _ in p.imap_unordered(convert, patient_ids):
            pbar.update(1)
        pbar.close()

    print("[Done]!")




if __name__ == '__main__':
    main()

[Done]!
